# Scan the Folders

In [ ]:
import csv,os

In [ ]:
rootdir = '/mnt/home/edureka_524533/EronEmails/'

In [ ]:
csv_columns = ['ID','Date','From','To','Subject','MimeVer','ContentType','ContentEncoding','FromName','ToName','CC','BCC','Folder','Origin','FileName','Body']

In [ ]:
csv_file = 'Emails.csv'

In [ ]:
from email.parser import Parser

In [ ]:
# Def a function to perform all the above steps in one go
def read_email_data(inpFile):
    bodyList=[]
    with open(inpFile,"r") as f:
        data = ''
        data = f.read()
        email = Parser().parsestr(data)
        messageIDList = email['Message-ID']
        dateList = email['Date']
        fromList = email['From']
        
        #toList.append(email['To'])
        ###
        if email['To']:
            toEmail = email['To']
            toEmail = toEmail.replace("\n", "")
            toEmail = toEmail.replace("\r", "")
            toEmail = toEmail.replace("\t", "")
            toEmail = toEmail.replace(" ", "")
            toEmail = toEmail.split(",")
            toList = ''.join(toEmail)
        else:
            toList = ''
        ###
        if email['Subject']:
            data = email['Subject']
            data = data.replace("\n","")
            data = data.replace("\r","")
            data = data.replace("\t","")
            subjectList = data
        else:
            subjectList = ''
        #subjectList = email['Subject']
        mimeVer = email['Mime-Version']
        conTypeList = email['Content-Type']
        conEncodList = email['Content-Transfer-Encoding']
        fromNameList = email['X-From']
        toNameList = email['X-To']
        ccList = email['X-cc']
        bccList = email['X-bcc']
        folderList = email['X-Folder']
        originList = email['X-Origin']
        filenameList = email['X-FileName']
        #bodyList = email.get_payload()
        bodyList.append(email.get_payload())
        ### Create a dictionary with key:email['item'] & value the corresponding data in it
        dataDic = {}
        dataDic['ID'] = messageIDList
        dataDic['Date'] = dateList
        dataDic['From'] = fromList
        dataDic['To']=toList
        dataDic['Subject']=subjectList 
        dataDic['MimeVer']=mimeVer
        dataDic['ContentType']=conTypeList
        dataDic['ContentEncoding']=conEncodList
        dataDic['FromName']=fromNameList
        dataDic['ToName']=toNameList
        dataDic['CC']=ccList
        dataDic['BCC']=bccList
        dataDic['Folder']=folderList
        dataDic['Origin']=originList
        dataDic['FileName']=filenameList
        dataDic['Body']=bodyList[:]
        return dataDic

In [ ]:
#open the csv file in append mode and write the header
with open(csv_file, 'w') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=csv_columns)
        writer.writeheader()
#Call the os.walk to scan through folder tree structure and reach each file to populate the lists
        for directory,subdirectory,filenames in os.walk(rootdir):
        #initialize_process()
            for filename in filenames:
            #print(os.path.join(directory, filename))
                dataDic = read_email_data(os.path.join(directory, filename))
                writer.writerow(dataDic)
csvfile.close()

In [ ]:
# load csv to HDFS using:
    # hdfs dfs -copyFromLocal 'EnronEmails.csv' 'Datasets/'

# Load data into Spark DataFrame

In [1]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName("Module 8 Case Study 2").getOrCreate()

In [5]:
emailDF = spark.read.csv("/user/edureka_524533/Datasets/Emails.csv",inferSchema=True,header=True)

In [6]:
#emailPD.head()
emailDF.printSchema()

root
 |-- ID: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- From: string (nullable = true)
 |-- To: string (nullable = true)
 |-- Subject: string (nullable = true)
 |-- MimeVer: string (nullable = true)
 |-- ContentType: string (nullable = true)
 |-- ContentEncoding: string (nullable = true)
 |-- FromName: string (nullable = true)
 |-- ToName: string (nullable = true)
 |-- CC: string (nullable = true)
 |-- BCC: string (nullable = true)
 |-- Folder: string (nullable = true)
 |-- Origin: string (nullable = true)
 |-- FileName: string (nullable = true)
 |-- Body: string (nullable = true)



# Display the top 10 high-frequency users based on weekly numbers of emails sent.

In [13]:
gpBySenderDF = emailDF.groupby('From').count()

In [14]:
from pyspark.sql.functions import col

In [15]:
top10EmailSenders = gpBySenderDF.orderBy(col('count').desc()).limit(10)

In [16]:
top10EmailSenders.show(truncate=False)

+--------------------------------+-----+
|From                            |count|
+--------------------------------+-----+
|lynn.blair@enron.com            |1112 |
|outlook.team@enron.com          |1030 |
|no.address@enron.com            |106  |
|john.buchanan@enron.com         |73   |
|ava.garcia@enron.com            |52   |
|michael.bodnar@enron.com        |38   |
|special@flowgo.com              |38   |
|newsletter@quickinspirations.com|37   |
|updates@send4fun.com            |32   |
|shelley.corman@enron.com        |27   |
+--------------------------------+-----+



# Extract top 20 keywords from the subject text for both for the top 10 high-frequency users and for the non - high frequency users.

In [17]:
#Create a DF containing subject line along with Sender
subjDF = emailDF.select('From','Subject')

In [18]:
bad_rows_df = subjDF.filter(subjDF['From'].isNull()|subjDF['Subject'].isNull()) 

In [19]:
bad_rows_df.show()

+--------------------+-------+
|                From|Subject|
+--------------------+-------+
|                null|   null|
|courtney.barker@e...|   null|
|michael.bodnar@en...|   null|
|karen.clapper@enr...|   null|
|  sales@webdesin.com|   null|
|sharon.brown@enro...|   null|
|outlook.team@enro...|   null|
|outlook.team@enro...|   null|
|outlook.team@enro...|   null|
|outlook.team@enro...|   null|
|outlook.team@enro...|   null|
|outlook.team@enro...|   null|
|outlook.team@enro...|   null|
|outlook.team@enro...|   null|
|outlook.team@enro...|   null|
|outlook.team@enro...|   null|
|outlook.team@enro...|   null|
|outlook.team@enro...|   null|
|outlook.team@enro...|   null|
|outlook.team@enro...|   null|
+--------------------+-------+
only showing top 20 rows



In [20]:
top10UsersDF = subjDF.groupby('From').count().orderBy(col('count').desc()).limit(10)

In [21]:
top10UsersDF.show()

+--------------------+-----+
|                From|count|
+--------------------+-----+
|lynn.blair@enron.com| 1112|
|outlook.team@enro...| 1030|
|no.address@enron.com|  106|
|john.buchanan@enr...|   73|
|ava.garcia@enron.com|   52|
|michael.bodnar@en...|   38|
|  special@flowgo.com|   38|
|newsletter@quicki...|   37|
|updates@send4fun.com|   32|
|shelley.corman@en...|   27|
+--------------------+-----+



In [22]:
top10NonHighUsersDF = subjDF.groupby('From').count().orderBy(col('count').asc()).limit(10)

In [23]:
top10NonHighUsersDF.show()

+--------------------+-----+
|                From|count|
+--------------------+-----+
|press.release@enr...|    1|
|newsletter@rigzon...|    1|
|elizabeth.bouldin...|    1|
|  sales@webdesin.com|    1|
|bob.d.johnson@mai...|    1|
|dhenderson1@pclie...|    1|
|thedesk@scudderpu...|    1|
|announcements.enr...|    1|
|ken.powers@enron.com|    1|
|patrick.brennan@e...|    1|
+--------------------+-----+



In [24]:
# Extract Subject line for each of the ten users 
subjTopDF = top10UsersDF.join(subjDF,on=['From'],how='left_outer')

In [25]:
subjTopDF.count()

2545

In [26]:
subjTopDF.printSchema()

root
 |-- From: string (nullable = true)
 |-- count: long (nullable = false)
 |-- Subject: string (nullable = true)



In [27]:
subNonHighTopDF = top10NonHighUsersDF.join(subjDF,on=['From'],how='left_outer')

In [28]:
subNonHighTopDF.printSchema()

root
 |-- From: string (nullable = true)
 |-- count: long (nullable = false)
 |-- Subject: string (nullable = true)



In [29]:
subNonHighTopDF.show()

+--------------------+-----+--------------------+
|                From|count|             Subject|
+--------------------+-----+--------------------+
|shari.stack@enron...|    1|Confirmation Temp...|
|lorna.pennicooke@...|    1|Testing the Servi...|
|phil.lowry@enron.com|    1|    RE: OneOk Letter|
|  sales@webdesin.com|    1|                null|
|ld.stephens@enron...|    1|RE: Mt. Jesus Dri...|
|bob.d.johnson@mai...|    1|          Borderline|
|kkeuter@ftenergy.com|    1|Northern Natural Gas|
|james.saunders@en...|    1|RE: TW/ENA compre...|
|frankie.adams@enr...|    1|RE: Hartley IA TB...|
|tom.halpin@enron.com|    1|Follow up on the ...|
+--------------------+-----+--------------------+



In [32]:
from pyspark.sql.functions import udf,explode,split

In [37]:
wordDF = subNonHighTopDF.withColumn('word', explode(split(col('Subject'), ' ')))\
    .groupBy('word')\
    .count()\
    .sort('count', ascending=False)\
    .limit(20)\
    .show()

+----------+-----+
|      word|count|
+----------+-----+
|     Enron|    2|
|       and|    2|
|       RE:|    2|
|       Mt.|    1|
|     Jesus|    1|
|      Drip|    1|
|     Oneok|    1|
|       FW:|    1|
|   2001-20|    1|
|          |    1|
|   Hartley|    1|
|   payment|    1|
|    Follow|    1|
|Borderline|    1|
|  Northern|    1|
|   Natural|    1|
|       Gas|    1|
|        IA|    1|
|       TBS|    1|
|        #1|    1|
+----------+-----+



In [38]:
# Now to extract the Top 20 keywords in for Top 10 Users

In [39]:
wordDF = subjTopDF.withColumn('word', explode(split(col('Subject'), ' ')))\
    .groupBy('word')\
    .count()\
    .sort('count', ascending=False)\
    .limit(20)\
    .show()

+----------+-----+
|      word|count|
+----------+-----+
|       FW:|  454|
|         -|  454|
|       RE:|  397|
|      Mtg.|  283|
|      room|  273|
|Conference|  220|
|   Meeting|  178|
|    EB4102|  168|
|       for|  164|
|    Oncall|  119|
|     Staff|  112|
|          |  106|
|    Weekly|  103|
|        in|   97|
|        to|   96|
|       Re:|   89|
|      Team|   85|
|        TW|   83|
|        on|   82|
|conference|   80|
+----------+-----+



# Extract top 10 keywords by identifying removing the common stop words

In [40]:
subjDF.printSchema()

root
 |-- From: string (nullable = true)
 |-- Subject: string (nullable = true)



In [41]:
from pyspark.ml.feature import StopWordsRemover

In [42]:
stopWords = StopWordsRemover.loadDefaultStopWords('english')

In [45]:
def remove_stopWords(message):
    if message is None:
        return ' '
    else:
        wordList = message.split(' ')
        messageEdit = [word for word in wordList if word not in stopWords]
        message = ' '.join(messageEdit)
        return message

In [46]:
udf_stopWEdit = udf(remove_stopWords)

In [47]:
subjCleanDF = (subjDF.select('*', udf_stopWEdit(subjDF['Subject']).alias('Subject_stopW')))

In [48]:
subjCleanDF.printSchema()

root
 |-- From: string (nullable = true)
 |-- Subject: string (nullable = true)
 |-- Subject_stopW: string (nullable = true)



In [49]:
subjCleanDF.show(5)

+--------------------+--------------------+--------------------+
|                From|             Subject|       Subject_stopW|
+--------------------+--------------------+--------------------+
|amy.fitzpatrick@e...|Fitness Club Reim...|Fitness Club Reim...|
|office.chairman@e...|Organisational An...|Organisational An...|
|enron.announcemen...|2000 Chairman's A...|2000 Chairman's A...|
|casey@mercatorpar...|         GOOD ADVICE|         GOOD ADVICE|
|office.chairman@e...|      Code of Ethics|         Code Ethics|
+--------------------+--------------------+--------------------+
only showing top 5 rows



In [50]:
wordDF = subjCleanDF.withColumn('word', explode(split(col('Subject_stopW'), ' ')))\
    .groupBy('word')\
    .count()\
    .sort('count', ascending=False)\
    .limit(10)\
    .show()

+----------+-----+
|      word|count|
+----------+-----+
|       RE:|  590|
|       FW:|  567|
|         -|  536|
|      Mtg.|  284|
|      room|  273|
|          |  243|
|Conference|  231|
|   Meeting|  225|
|    EB4102|  168|
|    Oncall|  120|
+----------+-----+



# Extend the stop words dictionary by adding your own stop words such as ‘ —‘

In [51]:
import string

In [52]:
def remove_punctuations(message):
    print(message)
    if message is None:
        return ' '
    else: 
        messageEdit = [char for char in message if char not in string.punctuation]
        message = ''.join(messageEdit)
        return message

In [53]:
udf_puncEdit = udf(remove_punctuations)

In [54]:
subjNoPuncDF = (subjCleanDF.select('*', udf_puncEdit(subjCleanDF['Subject_stopW']).alias('Subject_punc')))

In [55]:
subjNoPuncDF.show(5)

+--------------------+--------------------+--------------------+--------------------+
|                From|             Subject|       Subject_stopW|        Subject_punc|
+--------------------+--------------------+--------------------+--------------------+
|amy.fitzpatrick@e...|Fitness Club Reim...|Fitness Club Reim...|Fitness Club Reim...|
|office.chairman@e...|Organisational An...|Organisational An...|Organisational An...|
|enron.announcemen...|2000 Chairman's A...|2000 Chairman's A...|2000 Chairmans Award|
|casey@mercatorpar...|         GOOD ADVICE|         GOOD ADVICE|         GOOD ADVICE|
|office.chairman@e...|      Code of Ethics|         Code Ethics|         Code Ethics|
+--------------------+--------------------+--------------------+--------------------+
only showing top 5 rows



In [56]:
wordDF = subjNoPuncDF.withColumn('word', explode(split(col('Subject_punc'), ' ')))\
    .groupBy('word')\
    .count()\
    .sort('count', ascending=False)\
    .limit(10)\
    .show()

+----------+-----+
|      word|count|
+----------+-----+
|          |  974|
|        RE|  590|
|        FW|  571|
|       Mtg|  325|
|      room|  273|
|   Meeting|  236|
|Conference|  232|
|    EB4102|  169|
|       Gas|  125|
|    Oncall|  121|
+----------+-----+



# Introduce a new column label to identify new, replied, and forwarded messages.

In [83]:
forwarded = 'fwd:'
forward = 'fw:'
replied = 're:'

In [84]:
def checkMessageType(message):
    if message is None:
        return 'n'
    message = message.lower()
    messageList = message.split(' ')
    if (forwarded in messageList) or (forward in messageList):
        return 'f'
    elif replied in messageList:
        return 'r'
    else:
        return 'n'

In [85]:
from pyspark.sql.functions import udf

In [86]:
udf_emailType = udf(checkMessageType)

In [87]:
emailDF = (emailDF.select('*', udf_emailType(emailDF['Subject']).alias('Email_Type')))

In [88]:
def checkNewEmail(emailType):
    if emailType == 'n':#if new set indicator to 'Y
        result = 'Y'
    else:
        result = 'N'
    return result

In [89]:
def checkForwardedEmail(emailType):
    if emailType == 'f':
        result = 'Y'
    else:
        result = 'N'
    return result

In [90]:
def checkRepliedEmail(emailType):
    if emailType == 'r':
        result = 'Y'
    else:
        result = 'N'
    return result

In [91]:
udf_newEmail = udf(checkNewEmail)

In [92]:
udf_forwardedEmail = udf(checkForwardedEmail)

In [93]:
udf_repliedEmail = udf(checkRepliedEmail)

In [94]:
emailDF = (emailDF.select('*', udf_newEmail(emailDF['Email_Type']).alias('NewEmail')))

In [95]:
emailDF = (emailDF.select('*', udf_forwardedEmail(emailDF['Email_Type']).alias('ForwardedEmail')))

In [96]:
emailDF = (emailDF.select('*', udf_repliedEmail(emailDF['Email_Type']).alias('RepliedEmail')))

# Get the trend of the over mail activity using the pivot table from spark itself

In [97]:
emailDF.printSchema()

root
 |-- ID: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- From: string (nullable = true)
 |-- To: string (nullable = true)
 |-- Subject: string (nullable = true)
 |-- MimeVer: string (nullable = true)
 |-- ContentType: string (nullable = true)
 |-- ContentEncoding: string (nullable = true)
 |-- FromName: string (nullable = true)
 |-- ToName: string (nullable = true)
 |-- CC: string (nullable = true)
 |-- BCC: string (nullable = true)
 |-- Folder: string (nullable = true)
 |-- Origin: string (nullable = true)
 |-- FileName: string (nullable = true)
 |-- Body: string (nullable = true)
 |-- Email_Type: string (nullable = true)
 |-- NewEmail: string (nullable = true)
 |-- ForwardedEmail: string (nullable = true)
 |-- RepliedEmail: string (nullable = true)



In [98]:
emailDF['Date','From'].show(truncate=False)

+-------------------------------------+-----------------------------+
|Date                                 |From                         |
+-------------------------------------+-----------------------------+
|Tue, 22 Aug 2000 01:06:00 -0700 (PDT)|amy.fitzpatrick@enron.com    |
|Mon, 31 Jul 2000 11:07:00 -0700 (PDT)|office.chairman@enron.com    |
|Fri, 11 Aug 2000 18:20:00 -0700 (PDT)|enron.announcements@enron.com|
|Wed, 7 Jun 2000 16:17:00 -0700 (PDT) |casey@mercatorpartners.com   |
|Mon, 14 Aug 2000 13:19:00 -0700 (PDT)|office.chairman@enron.com    |
|Mon, 10 Jul 2000 10:50:00 -0700 (PDT)|enron.chairman@enron.com     |
|Wed, 9 Aug 2000 13:28:00 -0700 (PDT) |office.chairman@enron.com    |
|Sun, 21 May 2000 18:55:00 -0700 (PDT)|office.chairman@enron.com    |
|Wed, 17 May 2000 04:13:00 -0700 (PDT)|office.chairman@enron.com    |
|Wed, 23 Aug 2000 09:16:00 -0700 (PDT)|office.chairman@enron.com    |
|Mon, 24 Jan 2000 02:06:00 -0800 (PST)|office.chairman@enron.com    |
|Tue, 25 Jul 2000 08

In [99]:
badDF = emailDF.filter(emailDF['Date'].isNull()) 

In [100]:
#Fri, 4 Aug 2000 03:48:00 -0700 (PDT)
# Extract Day, Date, Month, Year   

In [101]:
from pyspark.sql.functions import regexp_extract, col

In [102]:
day_pattern='\w{3},'

In [103]:
emailDF = emailDF.withColumn('Day', regexp_extract(col('Date'), day_pattern, 0))

In [114]:
emailDF['Date','Day','Month','Year'].show()

+--------------------+----+-----+----+
|                Date| Day|Month|Year|
+--------------------+----+-----+----+
|Tue, 22 Aug 2000 ...|Tue,| Aug |2000|
|Mon, 31 Jul 2000 ...|Mon,| Jul |2000|
|Fri, 11 Aug 2000 ...|Fri,| Aug |2000|
|Wed, 7 Jun 2000 1...|Wed,| Jun |2000|
|Mon, 14 Aug 2000 ...|Mon,| Aug |2000|
|Mon, 10 Jul 2000 ...|Mon,| Jul |2000|
|Wed, 9 Aug 2000 1...|Wed,| Aug |2000|
|Sun, 21 May 2000 ...|Sun,| May |2000|
|Wed, 17 May 2000 ...|Wed,| May |2000|
|Wed, 23 Aug 2000 ...|Wed,| Aug |2000|
|Mon, 24 Jan 2000 ...|Mon,| Jan |2000|
|Tue, 25 Jul 2000 ...|Tue,| Jul |2000|
|Thu, 17 Aug 2000 ...|Thu,| Aug |2000|
|Fri, 28 Jul 2000 ...|Fri,| Jul |2000|
|Fri, 25 Aug 2000 ...|Fri,| Aug |2000|
|Tue, 29 Aug 2000 ...|Tue,| Aug |2000|
|Mon, 10 Jul 2000 ...|Mon,| Jul |2000|
|Fri, 4 Aug 2000 0...|Fri,| Aug |2000|
|Thu, 27 Jul 2000 ...|Thu,| Jul |2000|
|Wed, 16 Aug 2000 ...|Wed,| Aug |2000|
+--------------------+----+-----+----+
only showing top 20 rows



In [105]:
month_pattern=' \w{3} '

In [106]:
emailDF = emailDF.withColumn('Month', regexp_extract(col('Date'), month_pattern, 0))

In [107]:
year_pattern = '[1-2]\d{3}'

In [109]:
emailDF = emailDF.withColumn('Year', regexp_extract(col('Date'), year_pattern, 0))

In [110]:
emailDF.printSchema()

root
 |-- ID: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- From: string (nullable = true)
 |-- To: string (nullable = true)
 |-- Subject: string (nullable = true)
 |-- MimeVer: string (nullable = true)
 |-- ContentType: string (nullable = true)
 |-- ContentEncoding: string (nullable = true)
 |-- FromName: string (nullable = true)
 |-- ToName: string (nullable = true)
 |-- CC: string (nullable = true)
 |-- BCC: string (nullable = true)
 |-- Folder: string (nullable = true)
 |-- Origin: string (nullable = true)
 |-- FileName: string (nullable = true)
 |-- Body: string (nullable = true)
 |-- Email_Type: string (nullable = true)
 |-- NewEmail: string (nullable = true)
 |-- ForwardedEmail: string (nullable = true)
 |-- RepliedEmail: string (nullable = true)
 |-- Day: string (nullable = true)
 |-- Month: string (nullable = true)
 |-- Year: string (nullable = true)



In [111]:
emailDF.groupBy('From').pivot('Email_Type').count().show()

+--------------------+----+----+----+
|                From|   f|   n|   r|
+--------------------+----+----+----+
|alice.johnson@enr...|   1|   3|null|
| rick.kile@enron.com|   1|null|   1|
|announcements.enr...|null|   1|null|
|thedesk@scudderpu...|null|   1|null|
|dhenderson1@pclie...|null|   1|null|
|integrated.soluti...|null|   3|null|
|nancy.bagot@enron...|null|   3|null|
|   40enron@enron.com|null|   8|   1|
|ken.powers@enron.com|null|   1|null|
|maggie.matheson@e...|null|   4|null|
|    ipayit@enron.com|null|   2|null|
|  sales@webdesin.com|null|   1|null|
|michele.winckowsk...|null|   4|   7|
|elizabeth.bouldin...|null|   1|null|
|bob.d.johnson@mai...|null|   1|null|
|raetta.zadow@enro...|  10|  11|   6|
|e..anderson@enron...|   2|   1|   5|
|press.release@enr...|null|   1|null|
|newsletter@rigzon...|null|   1|null|
|   news@real-net.net|null|   3|null|
+--------------------+----+----+----+
only showing top 20 rows



In [112]:
badDF = emailDF.filter(emailDF['Year'].isNull()).show()

+----+----+----+----+-------+-------+-----------+---------------+--------+------+----+----+------+------+--------+--------------------+----------+--------+--------------+------------+----+-----+----+
|  ID|Date|From|  To|Subject|MimeVer|ContentType|ContentEncoding|FromName|ToName|  CC| BCC|Folder|Origin|FileName|                Body|Email_Type|NewEmail|ForwardedEmail|RepliedEmail| Day|Month|Year|
+----+----+----+----+-------+-------+-----------+---------------+--------+------+----+----+------+------+--------+--------------------+----------+--------+--------------+------------+----+-----+----+
|null|null|null|null|   null|   null|       null|           null|    null|  null|null|null|  null|  null|    null|['\x00\x00\x00\x0...|         n|       Y|             N|           N|null| null|null|
+----+----+----+----+-------+-------+-----------+---------------+--------+------+----+----+------+------+--------+--------------------+----------+--------+--------------+------------+----+-----+----+


In [113]:
emailDF.filter(emailDF['Year'].isNotNull()).groupBy('From').pivot('Email_Type').count().show()

+--------------------+----+----+----+
|                From|   f|   n|   r|
+--------------------+----+----+----+
|alice.johnson@enr...|   1|   3|null|
|thedesk@scudderpu...|null|   1|null|
| rick.kile@enron.com|   1|null|   1|
|announcements.enr...|null|   1|null|
|integrated.soluti...|null|   3|null|
|dhenderson1@pclie...|null|   1|null|
|nancy.bagot@enron...|null|   3|null|
|   40enron@enron.com|null|   8|   1|
|maggie.matheson@e...|null|   4|null|
|ken.powers@enron.com|null|   1|null|
|    ipayit@enron.com|null|   2|null|
|  sales@webdesin.com|null|   1|null|
|elizabeth.bouldin...|null|   1|null|
|michele.winckowsk...|null|   4|   7|
|raetta.zadow@enro...|  10|  11|   6|
|bob.d.johnson@mai...|null|   1|null|
|e..anderson@enron...|   2|   1|   5|
|press.release@enr...|null|   1|null|
|newsletter@rigzon...|null|   1|null|
|   news@real-net.net|null|   3|null|
+--------------------+----+----+----+
only showing top 20 rows



# Use k-means clustering to create 4 clusters from the extracted keywords.

In [57]:
from pyspark.ml.clustering import KMeans,KMeansModel

In [58]:
from pyspark.ml.feature import VectorAssembler

In [59]:
#Extract all words from subject line and order them in descending order
allWordsDF = subjNoPuncDF.withColumn('word', explode(split(col('Subject_punc'), ' ')))\
    .groupBy('word')\
    .count()\
    .sort('count', ascending=False)\
    .collect()

In [61]:
AllWordsDF= spark.createDataFrame(allWordsDF)

In [62]:
AllWordsDF.printSchema()

root
 |-- word: string (nullable = true)
 |-- count: long (nullable = true)



In [ ]:
# Requirement is to create 4 clusters so K=4

In [63]:
vc = VectorAssembler(inputCols=['count'],outputCol='features')

In [64]:
newVecDF = vc.transform(AllWordsDF)

In [65]:
newVecDF.show(5)

+----+-----+--------+
|word|count|features|
+----+-----+--------+
|    |  974| [974.0]|
|  RE|  590| [590.0]|
|  FW|  571| [571.0]|
| Mtg|  325| [325.0]|
|room|  273| [273.0]|
+----+-----+--------+
only showing top 5 rows



In [66]:
kmeans = KMeans(k=4,seed=1)

In [67]:
model = kmeans.fit(newVecDF.select('features'))

In [68]:
transformed = model.transform(newVecDF)

In [69]:
transformed.show(5)

+----+-----+--------+----------+
|word|count|features|prediction|
+----+-----+--------+----------+
|    |  974| [974.0]|         1|
|  RE|  590| [590.0]|         3|
|  FW|  571| [571.0]|         3|
| Mtg|  325| [325.0]|         3|
|room|  273| [273.0]|         3|
+----+-----+--------+----------+
only showing top 5 rows



In [73]:
centers = model.clusterCenters()

In [74]:
for center in centers:
    print(center)

[3.22195775]
[974.]
[67.34883721]
[371.16666667]


# Use LDA to generate 4 topics from the extracted keywords

In [75]:
from pyspark.ml.clustering import LDA

In [79]:
# Trains a LDA model.
lda = LDA(k=10, maxIter=10)
model1 = lda.fit(newVecDF)

In [80]:
# Describe topics.
topics = model.describeTopics(3)
print("The topics described by their top-weighted terms:")
topics.show(truncate=False)

The topics described by their top-weighted terms:
+-----+-----------+-----------+
|topic|termIndices|termWeights|
+-----+-----------+-----------+
|0    |[0]        |[1.0]      |
|1    |[0]        |[1.0]      |
|2    |[0]        |[1.0]      |
|3    |[0]        |[1.0]      |
|4    |[0]        |[1.0]      |
|5    |[0]        |[1.0]      |
|6    |[0]        |[1.0]      |
|7    |[0]        |[1.0]      |
|8    |[0]        |[1.0]      |
|9    |[0]        |[1.0]      |
+-----+-----------+-----------+



In [82]:
# Shows the result
transformed = model1.transform(newVecDF)
transformed.show(truncate=False)

+----------+-----+--------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|word      |count|features|topicDistribution                                                                                                                                                                                                         |
+----------+-----+--------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|          |974  |[974.0] |[1.0114433398680371E-4,1.0078016005676634E-4,1.0081246419437194E-4,1.0115524146413965E-4,1.0063162334282231E-4,1.0140054313467611E-4,1.0074216062689085E-4,1.0052439231682387E-4,1.005975471431424E-4,0.9990922115337335] |
|RE        |